# Fed-BioMed Researcher base example

This example uses MNIST dataset. Please check `README.md` file in `notebooks` directory for the instructions to load MNIST dataset and configure nodes.


In [1]:
from fedbiomed.researcher.environ import environ
environ["EXPERIMENTS_DIR"]

/Users/scansiz/miniforge3/envs/fbm-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/Users/scansiz/development/fedbiomed/fbm-researcher/var/experiments'

## Define an experiment model and parameters

Declare a torch training plan MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    
    # Defines and return model 
    def init_model(self, model_args):
        return self.Net(model_args = model_args)
    
    # Defines and return optimizer
    def init_optimizer(self, optimizer_args):
        return torch.optim.Adam(self.model().parameters(), lr = optimizer_args["lr"])
    
    # Declares and return dependencies
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms"]
        return deps
    
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = nn.Dropout(0.25)
            self.dropout2 = nn.Dropout(0.5)
            self.fc1 = nn.Linear(9216, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = F.relu(x)
            x = self.conv2(x)
            x = F.relu(x)
            x = F.max_pool2d(x, 2)
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = F.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)


            output = F.log_softmax(x, dim=1)
            return output

    def training_data(self):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.model().forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


### Model arguments and training arguments

In [2]:
from fedbiomed.common.metrics import MetricTypes
model_args = {}

training_args = {
    'loader_args': { 'batch_size': 48, }, 
    'optimizer_args': {
        "lr" : 1e-3
    },
    #'test_ratio' : 0.25,
    #'test_batch_size': 64,
    #'test_metric': MetricTypes.F1_SCORE,
    #'test_on_global_updates': True,
    #'test_on_local_updates': True,
    #'test_metric_args': {'average': 'marco'},
    # 'use_gpu': True,  # automatically falls back to cpu on nodes that don't support gpu
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

### Create and run the experiment

In [3]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None,
                save_breakpoints=True,
                )

2024-11-08 09:45:09,288 fedbiomed INFO - Starting researcher service...

2024-11-08 09:45:09,290 fedbiomed INFO - Waiting 3s for nodes to connect...

2024-11-08 09:45:10,244 fedbiomed DEBUG - Node: NODE_933f1657-1ddc-41b3-ba75-c20aad4aef89 polling for the tasks

2024-11-08 09:45:10,422 fedbiomed DEBUG - Node: NODE_46be388b-3c9b-4ede-9608-f99c519ae60b polling for the tasks

2024-11-08 09:45:12,294 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

...

2024-11-08 09:45:12,311 fedbiomed INFO - Node selected for training -> NODE_933f1657-1ddc-41b3-ba75-c20aad4aef89

2024-11-08 09:45:12,312 fedbiomed INFO - Node selected for training -> NODE_46be388b-3c9b-4ede-9608-f99c519ae60b

<function extract_symbols at 0x1050a4f70>


2024-11-08 09:45:12,324 fedbiomed DEBUG - Model file has been saved: /Users/scansiz/development/workdir/fbm-researcher/var/experiments/Experiment_0000/model_940e5556-5180-4433-bb06-54ca0a00c7ba.py

Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.


In [4]:
exp.run()

2024-11-08 09:45:15,502 fedbiomed INFO - Sampled nodes in round 0 ['NODE_933f1657-1ddc-41b3-ba75-c20aad4aef89', 'NODE_46be388b-3c9b-4ede-9608-f99c519ae60b']

<function extract_symbols at 0x1050a4f70>


2024-11-08 09:45:15,507 fedbiomed INFO - Sending request 
					 To: NODE_933f1657-1ddc-41b3-ba75-c20aad4aef89 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-11-08 09:45:15,509 fedbiomed INFO - Sending request 
					 To: NODE_46be388b-3c9b-4ede-9608-f99c519ae60b 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-11-08 09:45:15,548 fedbiomed DEBUG - Node: NODE_933f1657-1ddc-41b3-ba75-c20aad4aef89 polling for the tasks

...

2024-11-08 09:45:23,760 fedbiomed INFO - Nodes that successfully reply in round 1 ['NODE_933f1657-1ddc-41b3-ba75-c20aad4aef89', 'NODE_46be388b-3c9b-4ede-9608-f99c519ae60b']

<function extract_symbols at 0x1050a4f70>


2024-11-08 09:45:23,774 fedbiomed DEBUG - Model file has been saved: /Users/scansiz/development/workdir/fbm-researcher/var/experiments/Experiment_0000/model_94c360a5-e1d1-44ae-a589-07653aa471f5.py

2024-11-08 09:45:23,777 fedbiomed INFO - breakpoint number 1 saved at /Users/scansiz/development/workdir/fbm-researcher/var/experiments/Experiment_0000/breakpoint_0001

2

In [16]:
loaded_exp = Experiment.load_breakpoint()

2024-11-04 17:25:10,383 fedbiomed DEBUG - found json file containing states at                breakpoint_0001.json

<function extract_symbols at 0x104f10550>


2024-11-04 17:25:10,393 fedbiomed DEBUG - Model file has been saved: /Users/scansiz/development/test/my-researcher/var/experiments/Experiment_0003/model_d9a490d2-46db-477c-8e10-15ca5425b3c2.py

2024-11-04 17:25:10,416 fedbiomed INFO - Experimentation reload from None successful!

In [ ]:
loaded_exp.run_once(increase=True)

2024-11-04 17:25:54,612 fedbiomed INFO - Sampled nodes in round 4 ['NODE_d59683b3-c090-46b8-8d84-625ec29eed8d', 'NODE_4238258b-586d-4ed7-9a7e-47c0b35dd9d6']

<function extract_symbols at 0x104f10550>


2024-11-04 17:25:54,617 fedbiomed INFO - Sending request 
					 To: NODE_d59683b3-c090-46b8-8d84-625ec29eed8d 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-11-04 17:25:54,618 fedbiomed INFO - Sending request 
					 To: NODE_4238258b-586d-4ed7-9a7e-47c0b35dd9d6 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-11-04 17:25:54,659 fedbiomed DEBUG - Node: NODE_d59683b3-c090-46b8-8d84-625ec29eed8d polling for the tasks

...

2024-11-04 17:25:58,119 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_d59683b3-c090-46b8-8d84-625ec29eed8d 
					 Round 5 Epoch: 1 | Iteration: 90/100 (90%) | Samples: 4320/4800
 					 Loss: 0.251992 
					 ---------

2024-11-04 17:25:58,172 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_4238258b-586d-4ed7-9a7e-47c0b35dd9d6 
					 Round 5 Epoch: 1 | Iteration: 90/100 (90%) | Samples: 4320/4800
 					 Loss: 0.124467 
					 ---------

2024-11-04 17:25:58,522 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_d59683b3-c090-46b8-8d84-625ec29eed8d 
					 Round 5 Epoch: 1 | Iteration: 100/100 (100%) | Samples: 4800/4800
 					 Loss: 0.128850 
					 ---------

2024-11-04 17:25:58,549 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_4238258b-586d-4ed7-9a7e-47c0b35dd9d6 
					 Round 5 Epoch: 1 | Iteration: 100/100 (100%) | Samples: 4800/4800
 					 Loss: 0.092966 
					 ---------

2024-11-04 17:25:58,564 fedbiomed INFO - Nodes that successfully reply in round 4 ['NODE_d59683b3-c090-46b8-8d84-625ec29eed8d', 'NODE_4238258b-586d-4ed7-9a7e-47c0b35dd9d6']

In [22]:
import os
os.environ["FEDBIOMED_RESEARCHER_COMPONENT_ROOT"]

KeyError: 'FEDBIOMED_RESEARCHER_COMPONENT_ROOT'

In [ ]:
exp.run_once(increase=True)

Save trained model to file

In [ ]:
exp.training_plan().export_model('./trained_model')

#### Display results

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())
